In [12]:
import numpy as np

# 예시 데이터: FP16 형식의 입력 X와 가중치 W (행렬)
# FP16은 16비트 부동소수점으로, 메모리 사용량을 줄이기 위해 사용됩니다.
# 이 예시에서는 입력 행렬 X와 가중치 행렬 W가 주어져 있습니다.
X = np.array([[2, -1, -1], [0, 3, 2], [-1, -1, 0]], dtype=np.float16)
W = np.array([[-1, 0], [2, 0], [0, -2]], dtype=np.float16)

# 벡터-wise 양자화 함수
def vector_wise_quantize(matrix, bits=8):
    """
    주어진 행렬을 벡터 단위로 양자화하는 함수입니다.
    벡터 단위로 최대 절대값을 찾고, 그 값을 기준으로 8비트 정수 값으로 변환합니다.
    
    Args:
    - matrix: 부동소수점으로 표현된 행렬 (FP16)
    - bits: 양자화에 사용할 비트 수 (기본값은 8비트)
    
    Returns:
    - quantized_matrix: 양자화된 정수 행렬
    - C: 각 벡터의 최대값을 저장한 정규화 상수
    """
    # 각 벡터(행)에서 가장 큰 절대값을 찾습니다. 이를 통해 스케일링할 정규화 상수를 얻습니다.
    C = np.max(np.abs(matrix), axis=1, keepdims=True)
    
    # 벡터의 각 값을 그 벡터의 최대값으로 나누어 0에서 1 사이의 값을 얻습니다.
    # 그런 다음, 8비트로 표현할 수 있도록 127을 곱해 정수로 변환합니다.
    quantized_matrix = np.round((matrix / C) * (2**(bits-1) - 1)).astype(np.int8)
    
    # 양자화된 행렬과 정규화 상수를 반환합니다.
    return quantized_matrix, C

# 디퀀타이즈 함수 (양자화된 값을 다시 원래 값으로 복원)
def dequantize(quantized_matrix, Cx, Cw, bits=8):
    """
    양자화된 행렬을 다시 부동소수점 값으로 복원하는 함수입니다.
    
    Args:
    - quantized_matrix: 8비트 정수로 양자화된 행렬
    - Cx: 입력 행렬 X의 정규화 상수
    - Cw: 가중치 행렬 W의 정규화 상수
    - bits: 양자화에 사용된 비트 수 (기본값은 8비트)
    
    Returns:
    - dequantized_matrix: 복원된 FP16 행렬
    """
    # scale 값은 127 (2^7 - 1)입니다. 이는 8비트로 표현할 수 있는 최대 값입니다.
    scale = (2**(bits-1) - 1)
    
    # 양자화된 행렬을 다시 원래 값으로 복원하려면 정규화 상수 Cx와 Cw를 사용하여
    # 행렬의 각 값을 원래 범위로 스케일링합니다.
    dequantized_matrix = (quantized_matrix / (scale * scale)) * (Cx * Cw.T)
    
    # 복원된 행렬을 반환합니다.
    return dequantized_matrix

# 1. X와 W를 벡터 단위로 양자화
# 벡터-wise로 최대 절대값을 찾아 각 벡터를 정수 값으로 스케일링합니다.
X_q, Cx = vector_wise_quantize(X)
W_q, Cw = vector_wise_quantize(W.T)  # W의 열 별 최대값을 계산하기 위해 전치(transpose) 사용

# 양자화된 행렬을 출력하여 확인합니다.
print("Quantized X:", X_q)
print("Quantized W:", W_q.T)  # 가중치 원래 크기로 출력

# 2. 양자화된 행렬 곱셈 수행 (8비트 연산)
# 양자화된 X_q와 W_q 행렬을 곱합니다. 이 연산은 8비트 정수로 처리되어 메모리 사용량과 연산 시간을 절약합니다.
out_q = np.matmul(X_q, W_q.T.astype(np.int32))  # 가중치 원래 크기로 곱셈

print("Quantized Matmul Result (Out):", out_q)

# 3. 디퀀타이즈 수행 (정규화 상수 Cx, Cw로 복원)
# 양자화된 결과를 다시 원래 부동소수점 값으로 복원합니다.
# 이 과정에서 Cx와 Cw를 사용해 행렬 곱셈 결과를 스케일링합니다.
out_fp16 = dequantize(out_q, Cx, Cw)

print("Dequantized Output (FP16):", out_fp16)

# 4. outlier 처리 (혼합 정밀도 분해)
# outlier는 매우 큰 값으로, 양자화 과정에서 정확도가 떨어질 수 있는 값들입니다.
# 특정 threshold 이상인 값들은 8비트 양자화 대신 16비트 연산으로 처리합니다.
threshold = 1.0  # Threshold 값을 1.0으로 설정 (절대값이 1 이상인 값들을 outlier로 간주)
outliers_X = np.abs(X) > threshold
outliers_W = np.abs(W) > threshold

# outlier 값을 행렬 곱셈에서 따로 처리하기 위한 새로운 방법
# outlier 위치를 찾아서 해당 위치에서만 곱셈합니다.
outlier_X_vals = X[outliers_X]  # outlier로 간주되는 X의 값들
outlier_W_vals = W[outliers_W]  # outlier로 간주되는 W의 값들

# outlier 값들의 곱셈을 별도로 처리합니다. 이 경우 값의 차원을 맞춰야 합니다.
# reshape을 사용해 차원을 맞춘 후 곱셈을 수행합니다.
if len(outlier_X_vals) > 0 and len(outlier_W_vals) > 0:
    outliers_result = np.matmul(outlier_X_vals.reshape(-1, 1), outlier_W_vals.reshape(1, -1))
else:
    outliers_result = 0  # outlier가 없으면 0

print("Outlier Matmul Result (FP16):", outliers_result)

# 최종 결과는 디퀀타이즈된 값과 outlier 처리된 결과를 합산하여 얻습니다.
final_output = out_fp16 + outliers_result

print("Final Output (FP16):", final_output)


Quantized X: [[ 127  -64  -64]
 [   0  127   85]
 [-127 -127    0]]
Quantized W: [[ -64    0]
 [ 127    0]
 [   0 -127]]
Quantized Matmul Result (Out): [[-16256   8128]
 [ 16129 -10795]
 [ -8001      0]]
Dequantized Output (FP16): [[-4.03149606  2.01574803]
 [ 6.         -4.01574803]
 [-0.99212598  0.        ]]
Outlier Matmul Result (FP16): [[ 4. -4.]
 [ 6. -6.]
 [ 4. -4.]]
Final Output (FP16): [[ -0.03149606  -1.98425197]
 [ 12.         -10.01574803]
 [  3.00787402  -4.        ]]
